In [1]:
import pandas as pd
import dill
import pickle

from pythainlp import word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.externals import joblib

/home/nutorbit/miniconda3/envs/chatbot/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


In [2]:
df = pd.read_csv('./data/00_intent.csv')
df

,text,label
0,มีงานต้องทำ,createTask
1,มีการบ้านต้องส่ง,createTask
2,มีงาน,createTask
3,เตือนหน่อย,createTask
4,เตือน,createTask
5,ต้องทำงาน,createTask
6,ต้องทำยังไม่ได้ทำ,createTask
7,มีงานอะไรบ้าง,displayTask
8,งานอะไรยังไม่ทำ,displayTask
9,เหลืองานอะไรบ้าง,displayTask


In [3]:
df['text'] = df['text'].apply(lambda x: ' '.join(word_tokenize(x)))
df

,text,label
0,มี งาน ต้อง ทำ,createTask
1,มี การบ้าน ต้อง ส่ง,createTask
2,มี งาน,createTask
3,เตือน หน่อย,createTask
4,เตือน,createTask
5,ต้อง ทำงาน,createTask
6,ต้อง ทำ ยัง ไม่ ได้ ทำ,createTask
7,มี งาน อะไร บ้าง,displayTask
8,งาน อะไร ยัง ไม่ ทำ,displayTask
9,เหลือ งาน อะไร บ้าง,displayTask


In [4]:
c_vectorizer = CountVectorizer(lambda x: x.split(' '), ngram_range=(1,2))
text_feature = c_vectorizer.fit_transform(df['text']).todense()
df_text_feature = pd.DataFrame(text_feature,
                               columns=list(map(lambda x: f'text_{x}', range(text_feature.shape[1]))))
df_text_feature.head()

,text_0,text_1,text_2,text_3,text_4,text_5,text_6,text_7,text_8,text_9,...,text_35,text_36,text_37,text_38,text_39,text_40,text_41,text_42,text_43,text_44
0,0,0,1,1,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df_vectorize = pd.concat(
    [df, df_text_feature],
    axis=1
).drop('text', axis=1)
df_vectorize.head()

,label,text_0,text_1,text_2,text_3,text_4,text_5,text_6,text_7,text_8,...,text_35,text_36,text_37,text_38,text_39,text_40,text_41,text_42,text_43,text_44
0,createTask,0,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,createTask,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,createTask,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,createTask,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,createTask,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
train, test = train_test_split(df_vectorize, test_size=0.2, random_state=10)

In [7]:
model = XGBClassifier()
model.fit(train.drop('label', axis=1), train['label'])

XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=0, num_parallel_tree=1,
              objective='multi:softprob', random_state=0, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method=None, validate_parameters=False, verbosity=None)

In [8]:
model.score(test.drop('label', axis=1), test['label'])

1.0

In [9]:
test

,label,text_0,text_1,text_2,text_3,text_4,text_5,text_6,text_7,text_8,...,text_35,text_36,text_37,text_38,text_39,text_40,text_41,text_42,text_43,text_44
14,displayTask,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0
5,createTask,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,createTask,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,displayTask,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
7,displayTask,0,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [10]:
model.fit(df_vectorize.drop('label', axis=1), df_vectorize['label'])

joblib.dump(model, './models/01_model.pkl')
pickle._dump(c_vectorizer, open('./models/01_vectorizer.pkl', 'wb'))